In [1]:
import pandaRL
import gym
import numpy as np
import time
import os


In [2]:
env = gym.make('pandaPlay-v0')
env.render('human')
env.reset()

current_dir=/home/sholto/Desktop/AI/pandaRL/pandaRL/envs


/home/sholto/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


{'observation': array([-1.8580314e-02,  6.5309055e-02, -4.8573155e-02,  7.1021378e-01,
        -7.9463510e-04, -1.0080069e-03,  7.0398492e-01,  2.0000000e-02,
         8.3180934e-02, -4.5180023e-02,  1.1159238e-01, -7.0710701e-01,
         1.3967930e-06,  6.8110046e-07,  7.0710653e-01,  2.9342022e-02,
        -2.0075889e-02,  1.0194480e-01, -5.9285253e-01, -3.8457337e-01,
         3.8363054e-01,  5.9452236e-01], dtype=float32),
 'achieved_goal': array([ 8.3180934e-02, -4.5180023e-02,  1.1159238e-01, -7.0710701e-01,
         1.3967930e-06,  6.8110046e-07,  7.0710653e-01,  2.9342022e-02,
        -2.0075889e-02,  1.0194480e-01, -5.9285253e-01, -3.8457337e-01,
         3.8363054e-01,  5.9452236e-01], dtype=float32),
 'desired_goal': array([-0.11611623,  0.07967193, -0.01289167, -0.17151459,  0.02187894,
         0.06858361], dtype=float32),
 'controllable_achieved_goal': array([-0.01858031,  0.06530906, -0.04857315,  0.02      ], dtype=float32),
 'full_positional_state': array([-1.8580314e

In [3]:
%%time
for i in range(0,25):
    o,r,d,_ = env.step(env.action_space.sample())

CPU times: user 543 ms, sys: 427 µs, total: 543 ms
Wall time: 424 ms


In [17]:
np.random.random(1)*2*np.pi

array([3.17690607])

In [18]:
np.pi

3.141592653589793

In [19]:
np.sin(np.pi)

1.2246467991473532e-16

In [4]:
keys = ['obs', 'acts', 'desired_goals', 'achieved_goals', 'full_positional_states']
dataset_path = 'collected_data/pick_demos/'
def load_data(path, keys):
    reset_states = []
    dataset_dict = {key:[] for key in keys}
    for demo in os.listdir(path):
        traj = np.load(path+demo+'/data.npz')
        [dataset_dict[key].append(traj[key]) for key in keys]
        
        reset_states.append(path+demo+'/env_states/')
    
    for key in keys:
        dataset_dict[key] = np.array(dataset_dict[key])
    return dataset_dict, reset_states
    


In [6]:
examples, reset_states = load_data(dataset_path, keys)

In [7]:
obs = examples['obs']
acts = examples['acts']
goals = examples['desired_goals']
ags = examples['achieved_goals']
full_positional_states = examples['full_positional_states']
print(obs.shape)

(2, 30, 21)


In [9]:
idx = np.random.choice(len(obs))
resets = reset_states[idx]
traj_len = len(obs[idx])
env.panda.reset_goal_pos(ags[idx,-1,:])
env.p.restoreState(fileName=resets+str(0)+'.bullet')
for i in range(0, traj_len):
    env.step(acts[idx, i, :])
    #time.sleep(0.1)

error: Couldn't restore state.

In [10]:
resets+str(0)+'.txt'

'collected_data/pick_demos/0/env_states/0.txt'

In [21]:
idx = np.random.choice(len(obs))
traj = obs[idx,:,:]


env.reset(traj[1])


In [22]:
for i in range(len(traj)):
    env.reset(traj[i])
    #env.visualise_sub_goal(full_positional_states[idx, -1,:])
   
    
    time.sleep(0.02)

In [10]:
env.reset_goal_pos(np.array([0.1,0.1,0.1]))

# We need Determinism

In [23]:
open_gripper = np.array([0.04])
closed_gripper = np.array([0.01])

def go_above(env, obj_number):
    desired_position = env.panda.calc_environment_state()[obj_number]['pos'] + np.array([0, 0.05, 0])
    current_position = env.panda.calc_actor_state()['pos']
    action = np.concatenate([desired_position - current_position, env.panda.default_arm_orn, open_gripper])
    return action

def descend(env, obj_number):
    desired_position = env.panda.calc_environment_state()[obj_number]['pos'] + np.array([0, -0.015, 0])
    current_position = env.panda.calc_actor_state()['pos']
    action = np.concatenate([desired_position - current_position, env.panda.default_arm_orn, open_gripper])
    return action

def close(env, obj_number):
    desired_position = env.panda.calc_environment_state()[obj_number]['pos']
    current_position = env.panda.calc_actor_state()['pos']
    action = np.concatenate([desired_position - current_position, env.panda.default_arm_orn, closed_gripper])
    return action

def lift(env, obj_number):
    desired_position = env.panda.calc_environment_state()[obj_number]['pos']
    desired_position[1] +=  0.02
    current_position = env.panda.calc_actor_state()['pos']
    action = np.concatenate([desired_position - current_position, env.panda.default_arm_orn, closed_gripper])
    return action

def take_to(env, position):
    desired_position = position
    current_position = env.panda.calc_actor_state()['pos']
    action = np.concatenate([desired_position - current_position, env.panda.default_arm_orn, closed_gripper])*0.5
    return action


In [24]:
times = [1,2,3,4,6]
states  = [go_above, descend, close, lift, take_to]
p = env.panda.bullet_client
t = 0
state_pointer = 0
dt = 2/10

action_buff = []
observation_buff = []
desired_goals_buff = []
achieved_goals_buff = []
controllable_achieved_goal_buff = []
full_positional_state_buff = []
take_to_pos =  np.random.uniform(env.goal_lower_bound, env.goal_upper_bound)


In [51]:
def dumpStateToFile(file):
    for i in range(p.getNumBodies()):
        pos, orn = p.getBasePositionAndOrientation(i)
        linVel, angVel = p.getBaseVelocity(i)
        txtPos = "pos=" + str(pos) + "\n"
        txtOrn = "orn=" + str(orn) + "\n"
        txtLinVel = "linVel" + str(linVel) + "\n"
        txtAngVel = "angVel" + str(angVel) + "\n"
        file.write(txtPos)
        file.write(txtOrn)
        file.write(txtLinVel)
        file.write(txtAngVel)


In [74]:
import os

In [88]:
o = env.reset()
env.panda.reset_goal_pos(take_to_pos)
t = 0
state_pointer = 0

acts, obs, goals, ags , cagb,  fpsb = [], [], [], [], [], []
os.remove("saveFile.txt")
file = open("saveFile.txt", "w")
dumpStateToFile(file)
file.close()
os.remove("state.bullet")
p.saveBullet("state.bullet")
time.sleep(2)
while(t < times[state_pointer]):
    if state_pointer == 4:
        action = states[state_pointer](env,take_to_pos)
    else:
        action = states[state_pointer](env, obj_number = 0)


    o2, r, d, _ = env.step(action)
    if d:
        break
    acts.append(action), obs.append(o['observation']), goals.append(o['desired_goal']), ags.append(o2['achieved_goal']) ,\
    cagb.append(o2['controllable_achieved_goal']), fpsb.append(o2['full_positional_state'])
    o = o2

    t += dt
    if t >= times[state_pointer]:
        state_pointer += 1

    if t >=  times[-1]:
        if r > -1: # if demo was successful, append
            print('Storing Replay')
            action_buff.append(acts), observation_buff.append(obs), desired_goals_buff.append(goals), achieved_goals_buff.append(ags), \
            controllable_achieved_goal_buff.append(cagb), full_positional_state_buff.append(fpsb)
        break

Storing Replay


In [89]:
p.restoreState(fileName="state.bullet")

In [90]:
#env.reset(obs[0])
for i in range(len(acts)):
    
    env.panda.reset_goal_pos(take_to_pos)
    #env.visualise_sub_goal(full_positional_states[idx, -1,:])
   
    env.step(acts[i])
    time.sleep(0.05)

In [1]:
env.reset(obs[0])
i=0

NameError: name 'env' is not defined

In [3]:
import os
os.listdir('collected_data')

['pick.npz', 'panda_sim.py']

In [6]:
os.makedirs('collected_data/pick_demos')

In [49]:
env.step(acts[i])
i=i+1

In [1]:

import pybullet as p
import time
import pybullet_data
print(pybullet_data.getDataPath())
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

p.resetSimulation()
timinglog = p.startStateLogging(p.STATE_LOGGING_PROFILE_TIMINGS, "loadingBenchVR.json")
p.configureDebugVisualizer(p.COV_ENABLE_RENDERING, 0)
print("load plane.urdf")
p.loadURDF("plane.urdf")
print("load r2d2.urdf")

p.loadURDF("r2d2.urdf", 0, 0, 1)
print("load kitchen/1.sdf")
p.loadSDF(pybullet_data.getDataPath() + 'kitchens/1.sdf')
print("load 100 times plate.urdf")
for i in range(100):
  p.loadURDF("dinnerware/plate.urdf", 0, i, 1)

p.configureDebugVisualizer(p.COV_ENABLE_RENDERING, 1)

p.stopStateLogging(timinglog)
print("stopped state logging")
p.getCameraImage(320, 200)

while (1):
  p.stepSimulation()

/home/sholto/anaconda3/lib/python3.7/site-packages/pybullet_data
load plane.urdf
load r2d2.urdf
load kitchen/1.sdf


error: Cannot load SDF file.

In [5]:
import os
os.path.exists(pybullet_data.getDataPath() + '/kitchens/1.sdf')

True

In [4]:
pybullet_data.getDataPath() + 'kitchens/1.sdf'

'/home/sholto/anaconda3/lib/python3.7/site-packages/pybullet_datakitchens/1.sdf'

In [1]:
import pybullet as p
import time
import pybullet_data

p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.createCollisionShape(p.GEOM_PLANE)
p.createMultiBody(0, 0)

sphereRadius = 0.05
colSphereId = p.createCollisionShape(p.GEOM_SPHERE, radius=sphereRadius)
colBoxId = p.createCollisionShape(p.GEOM_BOX,
                                  halfExtents=[sphereRadius, sphereRadius*2, sphereRadius])

colBoxId = p.createCollisionShape(p.GEOM_MESH,fileName='duck_vhacd.obj')

mass = 0
visualShapeId = -1

link_Masses = [1]
linkCollisionShapeIndices = [colBoxId]
linkVisualShapeIndices = [-1]
linkPositions = [[0, 0.3, 0.11]]
linkOrientations = [[0, 0, 0, 1]]
linkInertialFramePositions = [[0, -0.3, -0.30]]
linkInertialFrameOrientations = [[1, 0, 0, 1]]
indices = [0]
jointTypes = [p.JOINT_REVOLUTE]
#jointTypes = [p.JOINT_PRISMATIC]
axis = [[1, 0, 0]]

i = 0
j= 0
k=0
basePosition = [
  0,0,0
]
baseOrientation = [0, 0, 0, 1]

sphereUid = p.createMultiBody(mass,
                              colBoxId,
                              visualShapeId,
                              basePosition,
                              baseOrientation,
                              linkMasses=link_Masses,
                              linkCollisionShapeIndices=linkCollisionShapeIndices,
                              linkVisualShapeIndices=linkVisualShapeIndices,
                              linkPositions=linkPositions,
                              linkOrientations=linkOrientations,
                              linkInertialFramePositions=linkInertialFramePositions,
                              linkInertialFrameOrientations=linkInertialFrameOrientations,
                              linkParentIndices=indices,
                              linkJointTypes=jointTypes,
                              linkJointAxis=axis)

p.changeDynamics(sphereUid,
               -1,
               spinningFriction=0.001,
               rollingFriction=0.001,
               linearDamping=0.0)
# for joint in range(p.getNumJoints(sphereUid)):
#     p.setJointMotorControl2(sphereUid, joint, p.VELOCITY_CONTROL, targetVelocity=1, force=10)

p.setGravity(0, 0, -10)
p.setRealTimeSimulation(1)

p.getNumJoints(sphereUid)
for i in range(p.getNumJoints(sphereUid)):
  p.getJointInfo(sphereUid, i)

# while (1):
#   for i in range(p.getNumJoints(sphereUid)):
#       print(p.getJointInfo(sphereUid, i))


  #time.sleep(0.01)

In [5]:
for i in range(p.getNumJoints(sphereUid)):
  print(p.getJointState(sphereUid, i))

(0.9794792285273164, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.8182566176524201)


In [4]:
for i in range(p.getNumJoints(sphereUid)):
  print(p.getJointState(sphereUid, i))

(2.055469357991168, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 4.052239232486693)


In [4]:
ori = p.getQuaternionFromEuler([0,np.pi/2,0])


In [5]:
linkOrientations =  [ori]
sphereUid = p.createMultiBody(mass,
                              colBoxId,
                              visualShapeId,
                              basePosition,
                              baseOrientation,
                              linkMasses=link_Masses,
                              linkCollisionShapeIndices=linkCollisionShapeIndices,
                              linkVisualShapeIndices=linkVisualShapeIndices,
                              linkPositions=linkPositions,
                              linkOrientations=linkOrientations,
                              linkInertialFramePositions=linkInertialFramePositions,
                              linkInertialFrameOrientations=linkInertialFrameOrientations,
                              linkParentIndices=indices,
                              linkJointTypes=jointTypes,
                              linkJointAxis=axis)

In [ ]:
p.quat_

In [1]:
import pybullet as p
from time import sleep
import pybullet_data

physicsClient = p.connect(p.GUI)

p.setAdditionalSearchPath(pybullet_data.getDataPath())

p.resetSimulation(p.RESET_USE_DEFORMABLE_WORLD)

p.setGravity(0, 0, -10)

planeId = p.loadURDF("plane.urdf", [0,0,-2])

boxId = p.loadURDF("cube_small.urdf", [0,3,2],useMaximalCoordinates = True)

bunnyId = p.loadSoftBody("torus.vtk", mass = 1, useNeoHookean = 1, NeoHookeanMu = 60, NeoHookeanLambda = 200, NeoHookeanDamping = 0.01, collisionMargin = 0.006, useSelfCollision = 1, frictionCoeff = 0.5, repulsionStiffness = 800)

bunny2 = p.loadURDF("torus_deform.urdf", [0,1,0], flags=p.URDF_USE_SELF_COLLISION)

p.setPhysicsEngineParameter(sparseSdfVoxelSize=0.25)
p.setRealTimeSimulation(1)

while p.isConnected():
  p.setGravity(0,0,-10)

TypeError: 'repulsionStiffness' is an invalid keyword argument for this function